# NLP Exercise

In [1]:
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
from tensorflow.keras import layers, Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

## Loading Dataset

In [2]:
train_df = pd.read_csv('train.csv')
train_data = train_df[['text', 'target']].sample(frac= 1.0, random_state= 42)
train_data

,text,target
2644,So you have a new weapon that can cause un-ima...,1
2227,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,Aftershock back to school kick off was great. ...,0
6845,in response to trauma Children of Addicts deve...,0
...,...,...
5226,@Eganator2000 There aren't many Obliteration s...,0
5390,just had a panic attack bc I don't have enough...,0
860,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,Officials say a quarantine is in place at an A...,1


In [3]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data['text'], train_data['target'], test_size= 0.1)
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [4]:
train_labels.value_counts()

target
0    3900
1    2951
Name: count, dtype: int64

## Load pretrained model

In [12]:
# Create Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
                                        input_shape= [],
                                        trainable= True,
                                        dtype= 'string',
                                        name= 'USE')

In [13]:
# Create model using Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation='relu', name= 'dense_layer_1'),
    layers.Dense(1, activation= 'sigmoid', name= 'output_layer')
], name= 'model_6_USE')

model_6.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.legacy.Adam(),
                metrics= ['accuracy'])

model_6.summary()

# inputs = layers.Input(shape= [], dtype= 'string', name= 'input_layer')
# x = sentence_encoder_layer(inputs)

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_layer_1 (Dense)       (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 256830721 (979.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
model_6.fit(train_sentences,
            train_labels,
            epochs= 5,
            validation_data= (val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 155s 682ms/step - loss: 0.4483 - accuracy: 0.7996 - val_loss: 0.4379 - val_accuracy: 0.8071
Epoch 2/5
215/215 [==============================] - 177s 822ms/step - loss: 0.2644 - accuracy: 0.8978 - val_loss: 0.5216 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 221s 1s/step - loss: 0.1749 - accuracy: 0.9299 - val_loss: 0.6258 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 481s 2s/step - loss: 0.1257 - accuracy: 0.9488 - val_loss: 0.6190 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 257s 1s/step - loss: 0.0616 - accuracy: 0.9736 - val_loss: 0.6553 - val_accuracy: 0.7559
